In [1]:
from pipelines.DataPipeline import DataPipeline
from transformers.Preprocessing import Preprocessing
from transformers.Rebalancer import Rebalancer
from utils.Common import Config
import pandas as pd


In [2]:
RAW_DATA_PATH = "../data/raw/KSI.csv"
df = pd.read_csv(RAW_DATA_PATH)

In [3]:
# fill missing values, adding new columns, extracting useful columns
pc = Preprocessing(df, Config.binary_columns, Config.cat_attribs, Config.num_attribs, Config.label )
new_df = pc.getFrame()

In [4]:
# seperate feature and label
X = new_df[Config.cat_attribs + Config.num_attribs+ Config.binary_columns]
Y = new_df[Config.label]

In [5]:
# pass feature to pipeline and convert it to numerical data
dp = DataPipeline(Config.num_attribs,Config.cat_attribs)
result = dp.process(X)

In [6]:
new_df.to_csv('../data/preprocessed/KSI_processed.csv', index=False)
result.to_csv('../data/preprocessed/KSI_processed_transformed.csv', index=False)

In [7]:
import joblib

joblib.dump(dp.col_transformer,'../models/datapipeline.pkl')


['../models/datapipeline.pkl']